In [11]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
from time import sleep
import csv

# mở chrome
driver = webdriver.Chrome()

# driver.get method() sẽ điều hướng đến một trang được cung cấp bởi địa chỉ URL
driver.get("https://linkedin.com/login")


# Logging into LinkedIn
time.sleep(2)
  
thongtin = open('thongtin.txt')
line = thongtin.readlines()
ten = line[0]
mk = line[1]
# định vị thành phần bằng id
username = driver.find_element("id", "username")
username.send_keys(ten)  # Enter Your Email Address
  
pword = driver.find_element("id", "password")
pword.send_keys(mk)        # Enter Your Password
  
login = driver.find_element("xpath", '//*[@id="organic-div"]/form/div[3]/button')
print('vy1-----------------')
login.click()

search_f = driver.find_element("xpath", '//*[@id="global-nav-typeahead"]/input')
print('vy2++++++++++++++++')
# tukhoa = input('nhap từ khóa:')
# search_f.send_keys("tukhoa")

search_f.send_keys("remote")
search_f.send_keys(Keys.RETURN)

driver.get("https://www.linkedin.com/jobs/search/?f_E=2&f_JT=F&geoId=92000000&keywords=remote&location=Worldwide&sortBy=R%2F")
time.sleep(5)

vy1-----------------


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="global-nav-typeahead"]/input"}
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0072F243]
	(No symbol) [0x006B7FD1]
	(No symbol) [0x005AD04D]
	(No symbol) [0x005DC0B0]
	(No symbol) [0x005DC22B]
	(No symbol) [0x0060E612]
	(No symbol) [0x005F85D4]
	(No symbol) [0x0060C9EB]
	(No symbol) [0x005F8386]
	(No symbol) [0x005D163C]
	(No symbol) [0x005D269D]
	GetHandleVerifier [0x009C9A22+2655074]
	GetHandleVerifier [0x009BCA24+2601828]
	GetHandleVerifier [0x007D8C0A+619850]
	GetHandleVerifier [0x007D7830+614768]
	(No symbol) [0x006C05FC]
	(No symbol) [0x006C5968]
	(No symbol) [0x006C5A55]
	(No symbol) [0x006D051B]
	BaseThreadInitThunk [0x76C36BD9+25]
	RtlGetFullPathName_UEx [0x77878FD2+1218]
	RtlGetFullPathName_UEx [0x77878F9D+1165]


In [82]:
def GetURL():
    driver.get(link)
    lastHeight = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        newHeight = driver.execute_script("return document.body.scrollHeight")
        if newHeight == lastHeight:
            break
        lastHeight = newHeight
    time.sleep(5)
    page_source = BeautifulSoup(driver.page_source, "html.parser")
    
    
    #info_div = page_source.find('div',{'class':'jobs-search-results display-flex flex-column'})
    info_div = page_source.find('ul',{'class':'jobs-search-results__list list-style-none'})
    sleep(2)

    profiles = info_div.find_all('a', class_ = 'disabled ember-view job-card-container__link')
    sleep(2)
    

    all_profile_URL = []
    for profile in profiles:
        profile_ID = profile.get('href')
        profile_URL = "https://www.linkedin.com" + profile_ID
        if profile_URL not in all_profile_URL:
            all_profile_URL.append(profile_URL)
        sleep(2)
    #print(all_profile_URL)
    return all_profile_URL

In [83]:
j=0
page=0
URLs_all_page = []
URL = "https://www.linkedin.com/jobs/search/?f_E=2&f_JT=F&geoId=92000000&keywords=remote&location=Worldwide&sortBy=R%2F"
for j in range(40):
    link = URL + "&start="+str(page)
    #print(link)
    URLs_one_page = GetURL()
    lastHeight = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        newHeight = driver.execute_script("return document.body.scrollHeight")
        if newHeight == lastHeight:
            break
        lastHeight = newHeight
    
    URLs_all_page = URLs_all_page + URLs_one_page
    page+=25

In [85]:
with open('output.csv', 'w', encoding='utf-8',  newline = '') as file_output:
    headers = ['Title', 'Name', 'Type','Time', 'Location' ,'Number of applying','URL' ]
    writer = csv.DictWriter(file_output, delimiter=',', lineterminator='\n',fieldnames=headers)
    writer.writeheader()
    for linkedin_URL in URLs_all_page:
        driver.get(linkedin_URL)
        #print('- Accessing profile: ', linkedin_URL)
        sleep(3)
        page_source = BeautifulSoup(driver.page_source, "html.parser")
        info_div = page_source.find('div',{'class':'p5'})
        try:
            ten_cong_viec = info_div.find('h1', class_='t-24 t-bold').get_text().strip() #Remove unnecessary characters 
            print('--- Tên công việc là: ', ten_cong_viec)
        except:
            ten_cong_viec = None
            print('--- Tên công việc là: ', ten_cong_viec)

        try:
            ten_cong_ty = info_div.find('a', class_='ember-view t-black t-normal').get_text().strip() 
            print('--- Tên công ty là: ', ten_cong_ty)
        except:
            ten_cong_ty = None
            print('--- Tên công ty là: ', ten_cong_ty)

        try:
            loai_cong_viec = info_div.find('div', class_='jobs-unified-top-card__job-insight').get_text().strip() 
            print('--- Loại công việc: ', loai_cong_viec)
        except:
            loai_cong_viec = None
            print('--- Loại công việc: ', loai_cong_viec)

        try:
            luong = info_div.find('a', class_='app-aware-link').get_text().strip() 
            print('--- Lương: ', luong)
        except:
            luong = None
            print('--- Lương: ', luong)

        try:
            vi_tri = info_div.find('span', class_='jobs-unified-top-card__bullet').get_text().strip() 
            print('--- Vị trí: ', vi_tri)
        except:
            vi_tri = None
            print('--- Vị trí: ', vi_tri)

        try:
            tg_dang  = info_div.find('span', class_='jobs-unified-top-card__posted-date').get_text().strip() 
            print('--- Thời gian đăng: ', tg_dang )
        except:
            tg_dang = None
            print('--- Thời gian đăng: ', tg_dang )

        try:
            nguoi_dang_ki = info_div.find('span', class_='jobs-unified-top-card__applicant-count').get_text().strip()
            print('--- Số lượng người đăng kí: ', nguoi_dang_ki)
        except:
            nguoi_dang_ki = None
            print('--- Số lượng người đăng kí: ', nguoi_dang_ki)

        writer.writerow({headers[0]:ten_cong_viec, headers[1]:ten_cong_ty, headers[2]:loai_cong_viec, headers[3]:tg_dang, headers[4]:vi_tri,headers[5]:nguoi_dang_ki, headers[6]:linkedin_URL})
        print('-----------------------------\n')
        
    

print('Nhiệm vụ hoàn thành!')

--- Tên công việc là:  Information Technology Engineering & Operations Specialist
--- Tên công ty là:  Brooks Running
--- Loại công việc:  Full-time · Entry level
--- Lương:  Try Premium for free
--- Vị trí:  Vietnam
--- Thời gian đăng:  1 week ago
--- Số lượng người đăng kí:  3 applicants
-----------------------------

--- Tên công việc là:  Python Software Engineer (Open Remote)
--- Tên công ty là:  Wizeline
--- Loại công việc:  Full-time · Entry level
--- Lương:  Try Premium for free
--- Vị trí:  Ho Chi Minh City, Ho Chi Minh City, Vietnam
--- Thời gian đăng:  2 weeks ago
--- Số lượng người đăng kí:  1 applicant
-----------------------------

--- Tên công việc là:  Web Application Developer, Upland (Remote) - $60,000/year USD
--- Tên công ty là:  Crossover for Work
--- Loại công việc:  Full-time · Entry level
--- Lương:  Try Premium for free
--- Vị trí:  Hanoi, Hanoi, Vietnam
--- Thời gian đăng:  2 days ago
--- Số lượng người đăng kí:  1 applicant
-----------------------------

--- 

KeyboardInterrupt: 

In [ ]:
def get_job_info():
  return  {
      'Tên công việc': '',
      'Tên công ty': None,
      'Vị trí': None,
      'Number of applying': None,
      'Title': '',
      'Description': {
          'Loại công vie':None,
          'Qualifications':None
          },
      
  }